# movie analysis -- audio

extract audio from video

In [9]:
import subprocess

# ffmpeg -i <infile> -ac 2 -f wav <outfile>
command = "ffmpeg -i ./Downfall.mp4 -ac 2 -f wav audio.wav"

subprocess.call(command, shell=True)

0

combine audio output

In [ ]:
command = "ffmpeg -i output.avi -i audio.wav -c:v copy -c:a aac -strict experimental output.mp4"

subprocess.call(command, shell=True)

split the audio by sclience to do analysis, chunk[i].wav is the name of audio chunks.

In [14]:
from pydub import AudioSegment
from pydub.silence import split_on_silence, detect_nonsilent

sound = AudioSegment.from_wav("audio.wav")

# consider it silent if quieter than -40 dBFS
nonsilent = detect_nonsilent(sound, min_silence_len = 300, silence_thresh = -40)
for i, (start_i, end_i) in enumerate(nonsilent):
    sound[max(0, start_i - 100) : end_i + 100].export("./audio_chunk/chunk{0}.wav".format(i), format="wav")

In [17]:
import glob
from python_speech_features.base import mfcc, fbank, logfbank, ssc
import scipy.io.wavfile as wav
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib
from sklearn.svm import SVC

clf2 = joblib.load('audio_SVC_model.pkl')
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)

cepCount = 13 #no of MFCC coefficients
nfeatures = 7 #features per coefficient

emotions = ["anger", "boredom", "disgust", "fear", "happy", "sadness", "neutral"]


def audio_read(datafs):
    (data, fs) = wav.read(datafs)
    ceps = mfcc(fs, numcep = cepCount)
    feat2 = ssc(fs, samplerate = 16000, winlen = 0.025, winstep=0.01, 
                nfilt = 26, nfft = 512, lowfreq = 0, highfreq = None, preemph=0.97)
    ls = []
    for i in range(ceps.shape[1]):
        temp = ceps[:,i]
        dtemp = np.gradient(temp)
        lfeatures  = [np.mean(temp), np.var(temp), np.amax(temp), np.amin(temp), 
        np.var(dtemp), np.mean(temp[0:temp.shape[0]/2]), np.mean(temp[temp.shape[0]/2:temp.shape[0]])]
        temp2 = np.array(lfeatures)
        ls.append(temp2)
    
    ls2 = []
    for i in range(feat2.shape[1]):
        temp = feat2[:,i]
        dtemp = np.gradient(temp)
        lfeatures = [np.mean(temp), np.var(temp), np.amax(temp), np.amin(temp), 
        np.var(dtemp), np.mean(temp[0:temp.shape[0]/2]), np.mean(temp[temp.shape[0]/2:temp.shape[0]])]
        temp2 = np.array(lfeatures)
        ls2.append(temp2)
    
    source = np.array(ls).flatten()
    source = np.append(source, np.array(ls2).flatten())
    return source

participants = glob.glob("./audio_chunk/*")
X = []
for data in participants:
    X.append(audio_read(data))
X = np.array(X)
X2 = scaler.fit_transform(X)
ind = clf2.predict(X2)

export the prediction by time, the csv file is like (start_time, stop_time, emotion). time in milesecond.

In [127]:
import csv
with open("audio_emotion.csv", "wb") as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    for i, (start_i, end_i) in enumerate(nonsilent):
        writer.writerow((start_i, end_i, emotions[ind[i]]))

In [2]:
import subprocess

command = "ffmpeg -i output.mp4 -i sub.srt \
-c copy -c:s mov_text \
-map 0 -map 1 -map_metadata 0 output2.mp4"
subprocess.call(command, shell=True)

0

In [50]:
from datetime import datetime, timedelta
subs = []
def covTime(msec):
    a = timedelta(milliseconds=msec)
    d = datetime.strptime(str(a), '%H:%M:%S.%f')
    return d.strftime("%H:%M:%S,%f")[:12]

for i, (start_i, end_i) in enumerate(nonsilent):
    print i+1
    print covTime(start_i)+ " --> " + covTime(end_i)
    print emotions[ind[i]]
    print
#     subs.append( ((covTime(start_i), covTime(end_i)), emotions[ind[i]]) )

1
00:00:00,412 --> 00:00:02,308
anger

2
00:00:03,922 --> 00:00:05,780
happy

3
00:00:06,393 --> 00:00:06,488
anger

4
00:00:08,127 --> 00:00:11,146
happy

5
00:00:12,164 --> 00:00:14,889
anger

6
00:00:17,788 --> 00:00:18,296
boredom

7
00:00:20,032 --> 00:00:20,204
happy

8
00:00:24,841 --> 00:00:25,003
anger

9
00:00:27,884 --> 00:00:28,098
happy

10
00:00:31,350 --> 00:00:33,330
boredom

11
00:00:34,360 --> 00:00:35,085
anger

12
00:00:55,318 --> 00:00:55,516
anger

13
00:01:01,045 --> 00:01:01,181
boredom

14
00:01:11,962 --> 00:01:12,043
happy

15
00:01:13,110 --> 00:01:14,378
disgust

16
00:01:15,276 --> 00:01:15,960
happy

17
00:01:16,276 --> 00:01:17,681
neutral

18
00:01:18,893 --> 00:01:19,651
anger

19
00:01:29,595 --> 00:01:30,227
boredom

20
00:01:30,547 --> 00:01:30,982
neutral

21
00:01:31,637 --> 00:01:33,640
happy

22
00:01:35,193 --> 00:01:37,817
neutral

23
00:01:38,185 --> 00:01:43,187
anger

24
00:01:43,649 --> 00:01:46,748
happy

25
00:01:47,414 --> 00:01:48,404


In [51]:
ffmpeg -i Downfall.mp4 -i sub.srt -c copy -c:s mov_text sub.mp4
https://www.youtube.com/watch?v=HOFFaiz6S4I

SyntaxError: invalid syntax (<ipython-input-51-7e8c658c735c>, line 1)

In [21]:
from datetime import datetime, timedelta
subs = []
def covTime(msec):
    a = timedelta(milliseconds=msec)
    d = datetime.strptime(str(a), '%H:%M:%S.%f')
    return d.strftime("%H:%M:%S,%f")[:12]

with open("audio_emotion.srt", "wb") as text_file:
    for i, (start_i, end_i) in enumerate(nonsilent):
        text_file.write(str(i+1)+"\n")
        text_file.write(covTime(start_i)+ " --> " + covTime(end_i) + "\n")
        text_file.writelines(emotions[ind[i]]+"\n")
        text_file.writelines("\n")


1
00:00:00,412 --> 00:00:02,308
anger

2
00:00:03,922 --> 00:00:05,780
happy

3
00:00:06,393 --> 00:00:06,488
anger

4
00:00:08,127 --> 00:00:11,146
happy

5
00:00:12,164 --> 00:00:14,889
anger

6
00:00:17,788 --> 00:00:18,296
boredom

7
00:00:20,032 --> 00:00:20,204
happy

8
00:00:24,841 --> 00:00:25,003
anger

9
00:00:27,884 --> 00:00:28,098
happy

10
00:00:31,350 --> 00:00:33,330
boredom

11
00:00:34,360 --> 00:00:35,085
anger

12
00:00:55,318 --> 00:00:55,516
anger

13
00:01:01,045 --> 00:01:01,181
boredom

14
00:01:11,962 --> 00:01:12,043
happy

15
00:01:13,110 --> 00:01:14,378
disgust

16
00:01:15,276 --> 00:01:15,960
happy

17
00:01:16,276 --> 00:01:17,681
neutral

18
00:01:18,893 --> 00:01:19,651
anger

19
00:01:29,595 --> 00:01:30,227
boredom

20
00:01:30,547 --> 00:01:30,982
neutral

21
00:01:31,637 --> 00:01:33,640
happy

22
00:01:35,193 --> 00:01:37,817
neutral

23
00:01:38,185 --> 00:01:43,187
anger

24
00:01:43,649 --> 00:01:46,748
happy

25
00:01:47,414 --> 00:01:48,404


In [11]:
text_file.write("1")

ValueError: I/O operation on closed file

In [7]:
file.write("",)

TypeError: descriptor 'write' requires a 'file' object but received a 'str'